# STRMaps

You can view the map at https://s3.amazonaws.com/filestogeaux.garyscorner.net/pub/STRMap.html FOR NOW
File will remain at s3://requesterpays.garyscorner.net/datasets/html/  (requester pays use AWS CLI [--request-payer requester])

Datasets:
* https://data.nola.gov/Housing-Land-Use-and-Blight/Map-of-Short-Term-Rental-Licenses/j5u3-2ueh

In [1]:
import pandas as pd
import folium
import numpy as np
import folium.plugins

from io import StringIO

from json import JSONEncoder
encoder = JSONEncoder().encode

from datetime import datetime

In [2]:
! curl https://data.nola.gov/api/views/en36-xvxg/rows.csv?accessType=DOWNLOAD -o DataSets\Short-Term_Rental_Permit_Applications.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  175k    0  175k    0     0   102k      0 --:--:--  0:00:01 --:--:--  102k
100 6989k    0 6989k    0     0  2569k      0 --:--:--  0:00:02 --:--:-- 2572k
100 9774k    0 9774k    0     0  2897k      0 --:--:--  0:00:03 --:--:-- 2902k


In [3]:
strDf = pd.read_csv("./DataSets/Short-Term_Rental_Permit_Applications.csv")

In [4]:
#drop without location
strDf = strDf[ strDf['X'] > 0.0]

In [5]:
strDf.drop(columns=['X','Y', 'Link'],inplace=True)

In [6]:
#drop dups and withdrawn
strDf = strDf[ ~(strDf['Current Status'].isin(['Duplicate','Withdrawn'])) ]

In [7]:
strDf['Current Status'].unique()

array(['Expired', 'Pending', 'Issued', 'Denied', 'Revoked'], dtype=object)

In [8]:
#this is done in javascript now
#strDf['Link'] = strDf['Link'].map(lambda url:  f"<a target='_blank' href='{url}'>One Stop App</a>")

In [9]:
%%time

def getPoints(val):
    try:
        
        lat,lon = val[1:-1].split(', ')
        lon = float(lon)
        lat = float(lat)
        pt = (lat,lon)
        
    except Exception as e:
        print(val)
        raise(e)
        
    return pt

strDf['Location'] = strDf['Location'].map(getPoints)


CPU times: total: 46.9 ms
Wall time: 70.5 ms


In [10]:
#now done in javascript
#strDf['Address'] = strDf.apply(lambda a: f"<a target='_blank' href='http://maps.google.com/maps?q=&layer=c&cbll={a['Location'][0]},{a['Location'][1]}'>{a['Address']}</a>",axis=1)

In [11]:
strDf['Current Status'].value_counts()

Current Status
Expired    16580
Denied      7760
Pending     3784
Issued      1265
Revoked      143
Name: count, dtype: int64

In [12]:
issueDates = pd.to_datetime(strDf['Issue_Date'])
issueDatesMax = issueDates.max()

(issueDatesMax, np.count_nonzero(issueDates == issueDatesMax))

(Timestamp('2024-07-11 00:00:00'), 7)

In [13]:
statusList = list(np.unique(strDf['Current Status']))

In [14]:
colMap = {
    'Guest Occupancy Limit':'Guest Limit',
    'License Holder Name':'License Holder',
    
}

strDf.rename(columns=colMap, inplace=True)

In [15]:
#for production
outputObj = {col:strDf[col].to_list() for col in strDf.columns}

In [16]:
config = {'lastUpdated':datetime.now().strftime('%A %B %m %Y')}

In [17]:
outputWrapper = {'config':config,'data':outputObj}

In [18]:
%%time

with open('STRMapTemplate.html', 'r') as f:
    templateHTML = f.read()

templateHTML.replace('%DATETIME%', str(datetime.now()))
    
beginHTML, endHTML = templateHTML.split('DATA_ALL')

with open('STRMap.html', 'w') as f:
    f.write(beginHTML)
    f.write(encoder(outputWrapper))
    f.write(endHTML)

CPU times: total: 125 ms
Wall time: 222 ms


In [19]:
%%time
!aws s3 cp STRMap.html s3://requesterpays.garyscorner.net/datasets/html/

Completed 256.0 KiB/7.9 MiB (56.2 KiB/s) with 1 file(s) remaining
Completed 512.0 KiB/7.9 MiB (67.8 KiB/s) with 1 file(s) remaining
Completed 768.0 KiB/7.9 MiB (76.8 KiB/s) with 1 file(s) remaining
Completed 1.0 MiB/7.9 MiB (81.5 KiB/s) with 1 file(s) remaining  
Completed 1.2 MiB/7.9 MiB (89.4 KiB/s) with 1 file(s) remaining  
Completed 1.5 MiB/7.9 MiB (93.1 KiB/s) with 1 file(s) remaining  
Completed 1.8 MiB/7.9 MiB (95.1 KiB/s) with 1 file(s) remaining  
Completed 2.0 MiB/7.9 MiB (98.9 KiB/s) with 1 file(s) remaining  
Completed 2.2 MiB/7.9 MiB (103.0 KiB/s) with 1 file(s) remaining 
Completed 2.5 MiB/7.9 MiB (106.5 KiB/s) with 1 file(s) remaining 
Completed 2.8 MiB/7.9 MiB (111.4 KiB/s) with 1 file(s) remaining 
Completed 3.0 MiB/7.9 MiB (115.9 KiB/s) with 1 file(s) remaining 
Completed 3.2 MiB/7.9 MiB (119.4 KiB/s) with 1 file(s) remaining 
Completed 3.5 MiB/7.9 MiB (122.9 KiB/s) with 1 file(s) remaining 
Completed 3.8 MiB/7.9 MiB (126.3 KiB/s) with 1 file(s) remaining 
Completed 

In [20]:
#https://s3.amazonaws.com/filestogeaux.garyscorner.net/pub/STRMap.html
!aws s3 cp s3://requesterpays.garyscorner.net/datasets/html/STRMap.html s3://filestogeaux.garyscorner.net/pub/ --acl public-read

Completed 7.9 MiB/7.9 MiB (20.9 MiB/s) with 1 file(s) remaining
copy: s3://requesterpays.garyscorner.net/datasets/html/STRMap.html to s3://filestogeaux.garyscorner.net/pub/STRMap.html


In [21]:
print(f"Finished {datetime.now()}")

Finished 2024-07-12 11:32:10.298019
